In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import knn

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df = pd.read_csv('data/heart_2020_small.csv')
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,28.15,No,No,No,7,0,Yes,Female,80 or older,White,No,No,Good,7,No,No,No
1,Yes,25.06,No,No,No,0,0,Yes,Female,80 or older,White,Yes,No,Good,7,No,No,Yes
2,No,23.67,Yes,No,Yes,0,0,No,Male,75-79,White,No,Yes,Very good,9,No,No,Yes
3,No,24.72,Yes,No,No,0,0,No,Female,65-69,Black,No,Yes,Very good,7,No,No,No
4,No,21.03,No,No,No,0,0,No,Female,80 or older,White,No,No,Very good,8,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,No,32.92,Yes,No,No,0,0,Yes,Female,55-59,Hispanic,Yes,No,Fair,6,No,No,No
3193,No,25.61,No,No,No,0,0,No,Female,25-29,Hispanic,No,Yes,Excellent,8,No,No,No
3194,No,33.72,No,No,No,0,0,No,Male,60-64,Hispanic,No,Yes,Very good,9,No,No,No
3195,No,29.76,No,No,No,0,0,No,Female,45-49,Hispanic,No,No,Very good,4,No,No,No


In [ ]:
individuals = pd.DataFrame(df.iloc[:,1:])
individuals

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,28.15,No,No,No,7,0,Yes,Female,80 or older,White,No,No,Good,7,No,No,No
1,25.06,No,No,No,0,0,Yes,Female,80 or older,White,Yes,No,Good,7,No,No,Yes
2,23.67,Yes,No,Yes,0,0,No,Male,75-79,White,No,Yes,Very good,9,No,No,Yes
3,24.72,Yes,No,No,0,0,No,Female,65-69,Black,No,Yes,Very good,7,No,No,No
4,21.03,No,No,No,0,0,No,Female,80 or older,White,No,No,Very good,8,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,32.92,Yes,No,No,0,0,Yes,Female,55-59,Hispanic,Yes,No,Fair,6,No,No,No
3193,25.61,No,No,No,0,0,No,Female,25-29,Hispanic,No,Yes,Excellent,8,No,No,No
3194,33.72,No,No,No,0,0,No,Male,60-64,Hispanic,No,Yes,Very good,9,No,No,No
3195,29.76,No,No,No,0,0,No,Female,45-49,Hispanic,No,No,Very good,4,No,No,No


In [ ]:
target = pd.DataFrame(df['HeartDisease'])

target

,SkinCancer
0,No
1,Yes
2,Yes
3,No
4,Yes
...,...
3192,No
3193,No
3194,No
3195,No


In [ ]:
# individuals.loc[:, 'Smoking'].map(dict([(k,i) for i,k in enumerate(individuals.loc[:, 'Smoking'].unique())]))
# individuals.dtypes[1]
type(individuals['Smoking'])

pandas.core.series.Series

In [ ]:
# Se sustituyen valores categóricos por valores numéricos

for col, t in zip(individuals.columns, individuals.dtypes):
    pos = 0
    keys = pd.DataFrame()    
    if (t == 'object'):
        key = dict([(k,i) for i,k in enumerate(individuals.loc[:, col].unique())])
        individuals[col] = individuals[col].map(key)

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(individuals, target, test_size=0.3)


In [ ]:
classifier = knn.Knn(1)
classifier.fit(xTrain, yTrain)
# classifier.predSingle(yTrain[0])

In [ ]:
classifier.predSingle(xTrain.iloc[1])

'No'

In [ ]:
# classifier.pred(xTest)

KeyboardInterrupt: 